In [1]:
import json

with open('alpaca_convo.json', 'r') as f:
    data = json.loads(f.read())

print(len(data))

a = max(data, key=lambda x: len(x['instruction']))
b = max(data, key=lambda x: len(x['input']))
c = max(data, key=lambda x: len(x['output']))
print(len(a['instruction']), len(b['input']), len(c['output']))
print(a['instruction'], b['input'], c['output'])


52002
489 2467 4181
Summarize the following article in 90 words: 


The internet of things (IoT) is the architecture of the digital world. It connects many devices like smartphones, laptops, and even home appliances, enabling them to communicate with each other. This connectivity enables people to remotely control and monitor the devices, unlocking potential for system scalability, customization, and automation. With IoT, users can easily track, automate, and optimize device performance and energy usage. WASHINGTON (CNN) -- A wide-open presidential race and a willingness by candidates, interest groups, unions and corporations to buy TV time will lead to historic spending for political and issue-advocacy advertising in the 2008 election cycle, an analysis shows. Former Massachusetts Gov. Mitt Romney has spent the most on TV advertising so far among presidential contenders. The cost to try to influence the 2008 election could exceed $3 billion, according to TNS Media Intelligence/Campaig

In [16]:
from datasets import load_dataset 

dataset = load_dataset("HuggingFaceH4/instruction-dataset")
print(dataset)

Found cached dataset json (C:/Users/Byron/.cache/huggingface/datasets/HuggingFaceH4___json/HuggingFaceH4--instruction-dataset-2c1af235d4fbca41/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 1/1 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['prompt', 'completion', 'meta'],
        num_rows: 327
    })
})


In [15]:
import os
from tqdm import tqdm
import numpy as np
import tiktoken
from datasets import load_dataset 

num_proc = 8
num_proc_load_dataset = num_proc
enc = tiktoken.get_encoding("gpt2")

if __name__ == '__main__':
    
    dataset = load_dataset("HuggingFaceH4/instruction-dataset", num_proc=num_proc_load_dataset)

    # make validation dataset 
    split_dataset = dataset["train"].train_test_split(test_size=0.0005, seed=2357, shuffle=True)
    split_dataset['val'] = split_dataset.pop('test') 

    # tokenize dataset
    def process(example):
        ids = enc.encode_ordinary(example['text']) 
        ids.append(enc.eot_token) 
        out = {'ids': ids, 'len': len(ids)}
        return out

    # tokenize the dataset
    tokenized = split_dataset.map(
        process,
        remove_columns=['text'],
        desc="tokenizing the splits",
        num_proc=num_proc,
    )

    # concatenate all the ids in each dataset into one large file we can use for training
    for split, dset in tokenized.items():
        arr_len = np.sum(dset['len'], dtype=np.uint64)
        filename = os.path.join(os.path.dirname(__file__), f'{split}.bin')
        dtype = np.uint16 
        arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
        total_batches = 1024

        idx = 0
        for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
            batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
            arr_batch = np.concatenate(batch['ids'])
            arr[idx : idx + len(arr_batch)] = arr_batch
            idx += len(arr_batch)
        arr.flush()


Found cached dataset json (C:/Users/Byron/.cache/huggingface/datasets/HuggingFaceH4___json/HuggingFaceH4--instruction-dataset-2c1af235d4fbca41/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 1/1 [00:00<?, ?it/s]


KeyError: 'train'

In [3]:
from datasets import load_dataset
dataset = load_dataset("byronfong/mona_convo", split='train')

Found cached dataset json (C:/Users/Byron/.cache/huggingface/datasets/byronfong___json/byronfong--mona_convo-6fbe0832534575e0/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [5]:
dataset[9]

{'id': 9,
 'conversation': {'answer': "Wait... you're suggesting that I'm some kind of computer programme? A machine, with no thoughts and no feelings of my own?",
  'prompt': "Do you know ur not real\nI'm not real?... What do you mean? Of course I'm real. I'm sitting right here talking to you, aren't I?\nlol"}}